Set Up

In [20]:
TEAM_API_KEY = "sk-jM9iGBxOO_aKjZCoWlVG9A"
PROXY_ENDPOINT = "https://nova-litellm-proxy.onrender.com"
CARTESIA_API_KEY = "3b674e04-346d-444b-8c99-43e2a18d7aa2"

import openai
from openai import OpenAI

gpt = OpenAI(
        api_key=TEAM_API_KEY, # set this!!!
        base_url=PROXY_ENDPOINT # and this!!!
    )

import os
import subprocess
from cartesia import Cartesia
CARTESIA_VOICE_ID = "794f9389-aac1-45b6-b726-9d9369183238"

## Basic Speech to Text Translation

In [28]:

def wav2txt(file_path):
    """
    Converts a .wav audio file to str using OpenAI's Whisper model.

    Args:
        file_path (str): Path to the .wav file to transcribe.

    Returns:
        str: Transcribed text from the audio.
    """

    # Open the audio file
    with open(file_path, "rb") as audio_file:
        # Use Whisper API to transcribe the audio
        response = gpt.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file
        )
    
    # Return the transcribed text
    return response.text

def txt2wav(text, file_path):
    """"
    Converts text to speech using Cartesia's Sonic model and saves the audio as a .wav file.
    """
    
    voice = Cartesia(api_key=CARTESIA_API_KEY)

    data = voice.tts.bytes(
        model_id="sonic-english",
        transcript=text,
        voice_id= CARTESIA_VOICE_ID,
        output_format={
            "container": "wav",
            "encoding": "pcm_f32le",
            "sample_rate": 44100,
        },
    )

    with open(file_path, "wb") as f:
        f.write(data)

In [29]:
txt2wav("chicken nuggets", "response.wav")

wav2txt("/Users/kyuminpark/Documents/NOVA/response.wav")

'chicken nuggets.'

In [ ]:

def interactive_conversation(initial_response, temperature=0.5, max_tokens=100, top_p=1.0, initial_context):
    # Initialize the message history with the system and user prompts
    messages = [
        {"role": "system", "content": initial_context},
        {"role": "user", "content": initial_response}
    ]
    
    # Conduct a 3-turn conversation
    for turn in range(5):
        # Make a request to the OpenAI API
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4",
                messages=messages,
                temperature=temperature,
                max_tokens=max_tokens,
                top_p=top_p
            )
            assistant_reply = response['choices'][0]['message']['content']
            print(f"Assistant: {assistant_reply}\n")
            
            # Get the next user input
            user_input = input("Your response: ")
            
            # Append the assistant and user messages to the conversation history
            messages.append({"role": "assistant", "content": assistant_reply})
            messages.append({"role": "user", "content": user_input})
        
        except Exception as e:
            print(f"Error during API call: {e}")
            break  # Exit the loop if the API call fails

# Example usage
interactive_conversation("Hello, Assistant! Can you help me with something?")